---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Xinyi Lu

---


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [84]:
import itertools

class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin #how many needed in a row to win
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        self.win = nwin #rule of the game: no. of moves by a player; fixed value
        
    def check_row(self, lst, n, move): #check for moves in the same row
        #lst: list of occ pieces incl. current move
        #n: no. of consecutive pieces
        sorted(lst, key=lambda x: x[1])
        #check if consecutive cols in same row
        for i in range(0, len(lst)):
            curr_lst = lst[i:i+n-1]
            if len(curr_lst) == n-1:
                curr_lst.append(move)
                col_no = list(map(lambda x: x[1], curr_lst))
                cols = self.check_consecutive(col_no)
                if cols:
                    return True
        return False
        
    def check_col(self, lst, n, move): #check for moves in the same col
        #lst: list of occ pieces
        #n: no. of consecutive pieces
        sorted(lst, key = lambda x: x[0]) #sort by row no.
        #check if consecutive rows in same col
        for i in range(0, len(lst)):
            curr_lst = lst[i:i+n-1]
            if len(curr_lst) == n-1:
                curr_lst.append(move)
                row_no = list(map(lambda x: x[0], curr_lst))
                rows = self.check_consecutive(row_no)
                if rows:
                    return True
        return False
            
    def check_diag(self, lst, n, move):
        diag_combi = list(itertools.combinations(lst, n-1)) 
        diag_combi = list(map(lambda x: list(x), diag_combi)) #convert list of tuples to list of lists
            
        for each in diag_combi:
            each.append(move)
            #check if rows and column numbers are consecutive
            diag_row = list(map(lambda x: x[0], each))
            diag_cols = list(map(lambda x: x[1], each))
                
            rows = self.check_consecutive(diag_row)
            cols = self.check_consecutive(diag_cols)
            if rows and cols:
                return True
        return False
        
    def check_consecutive(self, l):
        return sorted(l) == list(range(min(l), max(l)+1))
        
    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # if already occupied, return current state
        if move in state.board:
            return state
            
        new_utility = self.compute_utility(move, state)
        new_moves = state.moves
        new_moves.remove(move)
        new_state = State(new_moves)
        new_state.utility = new_utility
        new_state.board = state.board
        new_state.board[move] = state.to_move
        if state.to_move == 'R':
            new_state.to_move = 'B'
        else:
            new_state.to_move = 'R'
        
        ## need to change nwin too
        if self.nwin == self.win:
            self.nwin -= 1
        elif self.nwin != 1: #check for a few consecutive pieces in a row as long as not last move
            curr_moves = [move for move in new_state.board if new_state.board[move] == state.to_move]
            same_row = self.check_row(curr_moves, self.win-self.nwin+1, move)
            same_col = self.check_col(curr_moves, self.win-self.nwin+1, move)
            same_diag = self.check_diag(curr_moves, self.win-self.nwin+1, move)
            if same_row or same_col or same_diag: #there are a few consecutive moves in a row
                self.nwin -= 1

        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''
        if self.nwin != 1: #if there are >=2 moves needed to win the game
            return 0
        #get all the pieces for current state
        player = state.to_move
        lst = []
        for piece in state.board:
            occ_player = state.board[piece]
            if occ_player == player:
                lst.append(piece)
                
        #check for same column pieces
        same_col = list(filter(lambda x: x[1] == move[1], lst))
        if len(same_col) >= 3:  
            output = self.check_col(same_col, 4, move)
            if output and player == 'R':
                self.nwin = 0
                return 1
            elif output and player == 'B':
                self.nwin = 0
                return -1
            
        #check for same row pieces
        same_row = list(filter(lambda x: x[0] == move[0], lst))
        if len(same_row) >= 3:
            output = self.check_row(same_row, 4, move)
            if output and player == 'R':
                self.nwin = 0
                return 1
            elif output and player == 'B':
                self.nwin = 0
                return -1
            
        #check for diagonal row pieces
        #add pieces whose row/col number are <=3 from current move
        diag = list(filter(lambda x: (abs(x[0] - move[0])<=3) and (abs(x[1] - move[1])<=3), lst))
        if len(diag) >= 3: #if there are more than 3 pieces that are within 3 steps away from move
            output = self.check_diag(diag, 4, move)
            if output and player == 'R':
                self.nwin = 0
                return 1
            elif output and player == 'B':
                self.nwin = 0
                return -1
        return 0        

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        
        return (state.utility!=0) or (len(state.moves)==0)

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        
        # your code goes here...
        if player == 'R': #if player is Red
            return self.state.utility
        return -1*self.state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        
        # your code goes here...
        #player1 & player2 are moves
        #let player1 be move by 'R'
        #let player2 be move by 'B'
        
        limit = self.nrow * self.ncol
        turn = 0
        while turn <= limit:
            for player in (player1, player2):
                turn += 1
                move = player(self)
                if move is None: 
                    return self.state.utility
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    return self.state.utility
        return self.state.utility

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [85]:
import random

def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    state = game.state
    if len(state.board) == 0: #first move of the game
        #choose a move that is from the last row of the board
        pos_moves = list(filter(lambda x: x[0] == game.nrow, state.moves))
        return random.choice(pos_moves)
    
    #other moves of the game
    max_row = max(state.moves, key=lambda x: x[0])[0] #find max row number
    pos_moves = list(filter(lambda x: x[0] == max_row, state.moves)) #all available pieces in lowest non fully occupied row in the board
    # add moves from 1 row above occupied pieces
    occ = list(state.board.keys())
    for move in occ:
        new_row = move[0] - 1
        if new_row >= 1:
            pos_moves.append((new_row, move[1]))
    
    return random.choice(pos_moves)

We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [86]:
game = ConnectFour()
output = game.play_game(random_player, random_player)
print(output)

0


In [64]:
# 1000 games between two random players
results = []

for i in range(1000):
    game = ConnectFour()
    output = game.play_game(random_player, random_player)
    results.append(output)

print(results.count(1)/1000) #proportion of games that first player has won
print(results.count(0)/1000) #proportion of games that it is a draw
print(results.count(-1)/1000) #proportion of games that first player has lost

0.522
0.102
0.376


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [58]:
# 1000 games between two random players
results = []

for i in range(1000):
    game = ConnectFour(10, 10, 6)
    output = game.play_game(random_player, random_player)
    results.append(output)

print(results.count(1)/1000) #proportion of games that first player has won
print(results.count(0)/1000)
print(results.count(-1)/1000)

0.494
0.019
0.487


The decrease in wins turned into more losses for the first player and fewer draws.
This is because the state space has increased by a substantial amount, which makes it impossible to end the game in a draw.

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [79]:
# Your code here
def max_value(game, state, alpha, beta):
    player = state.to_move
    if game.game_over(state):
        return game.utility(state, player)
    value = -1*float("inf")
    for move in state.moves:
        value = max(value, min_value(game, game.result(move, state), alpha, beta))
        if value >= beta: 
            return value
        alpha = max(value, alpha)
    return value

def min_value(game, state, alpha, beta):
    player = state.to_move
    if game.game_over(state):
        return game.utility(state, player)
    value = float("inf")
    for move in state.moves:
        value = min(value, max_value(game, game.result(move, state), alpha, beta))
        if value <= alpha: 
            return value
        beta = min(value, beta)
    return value

def alphabeta_player(game):
    alpha = -1*float("inf")
    beta = float("inf")
    value = max_value(game, game.state, alpha, beta)
    for move in game.state.moves:
        if game.compute_utility(move, curr_state) == value:
            return move
    return
    #returns the minimax move

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [67]:
# Your code here
#using random
results_1 = []

for i in range(10):
    game = ConnectFour()
    output = game.play_game(random_player, random_player)
    results_1.append(output)

print(results_1.count(1)/10) #proportion of games that first player has won
print(results_1.count(0)/10) #draw
print(results_1.count(-1)/10) #lost

0.3
0.2
0.5


In [83]:
game = ConnectFour()
output = game.play_game(alphabeta_player, alphabeta_player)
print(output)
game.display()

0
R B R B R B R 
B R B R B R B 
R B R B R B R 
B R B B R B R 
B R B R B R B 
R B R B R B R 


In [87]:
#using ab pruning
results_2 = []

for i in range(10):
    game = ConnectFour()
    output = game.play_game(alphabeta_player, alphabeta_player)
    results_2.append(output)

print(results_2.count(1)/10) #proportion of games that first player has won
print(results_2.count(0)/10) #draw
print(results_2.count(-1)/10) #lost

0.0
1.0
0.0


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [ ]:
# Your code here.

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [ ]:
# Your code here.